In [26]:
import re
import os
import pandas as pd
import librosa
from pandarallel import pandarallel

pandarallel.initialize(nb_workers=4, progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [31]:
path = "/data/codes/gopt/egs/librispeech/s5/data/metadata/elsa-train.csv"
df = pd.read_csv(path, names=["path", "text"], sep="|")
df.head()

,path,text
0,/data/codes/gopt/egs/librispeech/s5/data/metad...,I THINK I WILL LIVE IN A IS A CAPITAL OF IN BE...
1,/data/codes/gopt/egs/librispeech/s5/data/metad...,IT'S LIKE IT NOT FAR AWAY FROM THE THE CENTER ...
2,/data/codes/gopt/egs/librispeech/s5/data/metad...,DEFINITELY YES BECAUSE LIKE MY HOUSE NEAR TO N...
3,/data/codes/gopt/egs/librispeech/s5/data/metad...,MANY SUPERMARKET MANY GROCERY STORE AND SOME E...
4,/data/codes/gopt/egs/librispeech/s5/data/metad...,AND SO ON


In [32]:
def get_duration(path):
    wave, sr = librosa.load(path, sr=16000)

    return wave.shape[0] / sr

duration = df.path.parallel_apply(get_duration)
duration.sum()/3600

605.0858940277778

In [23]:
df["utt_id"] = df.path.apply(lambda x: os.path.basename(x).split(".wav")[0])
df["spk_id"] = ['0'*(8-len(str(spk_id))) + f'{spk_id}' for spk_id in range(df.shape[0])]

In [24]:
def create_text_file(f, first_column, second_column):
    line = f'{first_column}\t{second_column}'
    f.write(line + "\n")

data_dir = "/data/codes/gopt/egs/librispeech/s5/data/prep/test_deepgram"

wavscp_path = f'{data_dir}/wav.scp'
text_path = f'{data_dir}/text'
spk2utt_path = f'{data_dir}/spk2utt'
utt2spk_path = f'{data_dir}/utt2spk'

with open(wavscp_path, "w", encoding="utf-8") as f:
    df.sort_values("utt_id").apply(lambda x: create_text_file(f, x["utt_id"], x["path"]), axis=1)
    
with open(text_path, "w", encoding="utf-8") as f:
    df.sort_values("utt_id").apply(lambda x: create_text_file(f, x["utt_id"], x["text"]), axis=1)
    
with open(spk2utt_path, "w", encoding="utf-8") as f:
    df.sort_values("utt_id").apply(lambda x: create_text_file(f, x["utt_id"], x["utt_id"]), axis=1)
    
with open(utt2spk_path, "w", encoding="utf-8") as f:
    df.sort_values("utt_id").apply(lambda x: create_text_file(f, x["utt_id"], x["utt_id"]), axis=1)